In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
model_path = "/content/drive/MyDrive/Q-Align_2/checkpoints"

In [ ]:
!git clone https://github.com/Q-Future/Q-Align.git
%cd Q-Align
!pip install -e ".[train]"
!pip install flash_attn --no-build-isolation

Cloning into 'Q-Align'...
remote: Enumerating objects: 745, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 745 (delta 3), reused 2 (delta 1), pack-reused 732
Receiving objects: 100% (745/745), 42.06 MiB | 12.62 MiB/s, done.
Resolving deltas: 100% (440/440), done.
Updating files: 100% (177/177), done.
/content/Q-Align
Obtaining file:///content/Q-Align
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.9/809.9 kB 48.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
 

In [ ]:
q_align_path = "/content/Q-Align"
external_checkpoints_path = "/content/drive/MyDrive/Q-Align_2/checkpoints"  # Adjust this path as needed
external_vqa_eval_path = "/content/drive/MyDrive/Q-Align_2/q_align/evaluate/vqa_eval.py"  # Adjust this path as needed

In [ ]:
external_checkpoints_path

'/content/drive/MyDrive/Q-Align_2/checkpoints'

In [ ]:
!mkdir -p {q_align_path}/checkpoints
!cp -r {external_checkpoints_path}/* {q_align_path}/checkpoints/
!cp {external_vqa_eval_path} {q_align_path}/q_align/evaluate/

In [ ]:
# Create directory for video data
!mkdir -p /content/video_data

# Copy video files
!cp /content/drive/MyDrive/Q-Align_data/*.mp4 /content/video_data/

In [ ]:
%%writefile /content/Q-Align/q_align/evaluate/vqa_eval.py
import torch
from PIL import Image
from q_align.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from q_align.conversation import conv_templates
from q_align.model.builder import load_pretrained_model
from q_align.mm_utils import process_images, tokenizer_image_token, get_model_name_from_path
import numpy as np

def load_video(video_file):
    from decord import VideoReader
    vr = VideoReader(video_file)
    fps = vr.get_avg_fps()
    frame_indices = [int(fps * i) for i in range(int(len(vr) / fps))]
    frames = vr.get_batch(frame_indices).asnumpy()
    return [Image.fromarray(frames[i]) for i in range(int(len(vr) / fps))]

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

def wa5(logits):
    logprobs = np.array([logits["excellent"], logits["good"], logits["fair"], logits["poor"], logits["bad"]])
    probs = np.exp(logprobs) / np.sum(np.exp(logprobs))
    return np.inner(probs, np.array([1, 0.75, 0.5, 0.25, 0]))


# Added new method for video score (Manisha)
def assess_video_quality(video_path, model_path, model_base=None, device="cuda"):
    # Load model
    model_name = get_model_name_from_path(model_path)
    tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, model_base, model_name, False, False, device=device)

    # Prepare input
    conv_mode = "mplug_owl2"
    inp = "How would you rate the quality of this video?"
    conv = conv_templates[conv_mode].copy()
    inp = inp + "\n" + DEFAULT_IMAGE_TOKEN
    conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt() + " The quality of the video is"

    toks = ["good", "poor", "high", "fair", "low", "excellent", "bad", "fine", "moderate", "decent", "average", "medium", "acceptable"]
    ids_ = [id_[1] for id_ in tokenizer(toks)["input_ids"]]

    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(device)

    # Process video
    image = load_video(video_path)
    image = [expand2square(img, tuple(int(x*255) for x in image_processor.image_mean)) for img in image]
    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().to(device)

    # Run inference
    with torch.inference_mode():
        output_logits = model(input_ids, images=[image_tensor])["logits"][:,-1]
        logits = {tok: output_logits.mean(0)[id_].item() for tok, id_ in zip(toks, ids_)}
        score = wa5(logits)

    return score, logits



Overwriting /content/Q-Align/q_align/evaluate/vqa_eval.py


In [ ]:
import sys
sys.path.append('/content/Q-Align')

from q_align.evaluate.vqa_eval import assess_video_quality

video_path = "/content/video_data/i2v (17).mp4"
model_path = "/content/Q-Align/checkpoints"

score, logits = assess_video_quality(video_path, model_path)
print(f"Video quality score: {score}")
print(f"Logits: {logits}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Video quality score: 0.24363672221323268
Logits: {'good': 12.9609375, 'poor': 15.6171875, 'high': 4.81640625, 'fair': 14.6171875, 'low': 5.875, 'excellent': 12.09375, 'bad': 15.1875, 'fine': 3.708984375, 'moderate': 2.896484375, 'decent': 3.765625, 'average': 4.125, 'medium': 3.080078125, 'acceptable': 3.03125}


In [ ]:
# %%writefile /content/Q-Align/q_align/evaluate/vqa_eval.py
# import argparse
# import torch

# from q_align.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
# from q_align.conversation import conv_templates, SeparatorStyle
# from q_align.model.builder import load_pretrained_model
# from q_align.mm_utils import process_images, tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

# from PIL import Image

# import requests
# from PIL import Image
# from io import BytesIO
# from transformers import TextStreamer


# from scipy.stats import spearmanr, pearsonr

# import json
# from tqdm import tqdm
# from collections import defaultdict

# import os

# def wa5(logits):
#     import numpy as np
#     logprobs = np.array([logits["excellent"], logits["good"], logits["fair"], logits["poor"], logits["bad"]])
#     probs = np.exp(logprobs) / np.sum(np.exp(logprobs))
#     return np.inner(probs, np.array([1,0.75,0.5,0.25,0.]))



# def disable_torch_init():
#     """
#     Disable the redundant torch default initialization to accelerate model creation.
#     """
#     import torch
#     setattr(torch.nn.Linear, "reset_parameters", lambda self: None)
#     setattr(torch.nn.LayerNorm, "reset_parameters", lambda self: None)


# def load_video(video_file):
#     from decord import VideoReader
#     vr = VideoReader(video_file)

#     # Get video frame rate
#     fps = vr.get_avg_fps()

#     # Calculate frame indices for 1fps
#     frame_indices = [int(fps * i) for i in range(int(len(vr) / fps))]
#     frames = vr.get_batch(frame_indices).asnumpy()
#     return [Image.fromarray(frames[i]) for i in range(int(len(vr) / fps))]


# def main(args):
#     # Model
#     disable_torch_init()

#     model_name = get_model_name_from_path(args.model_path)
#     tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name, args.load_8bit, args.load_4bit, device=args.device)


#     import json


#     image_paths = [
#         "/content/video_data/"
#         # "playground/data/",
#         # "playground/data/",
#         # "playground/data/KoNViD_1k_videos/",
#         # "playground/data/maxwell/",

#     ]

#     json_prefix = "playground/data/test_jsons/"
#     jsons = [
#         json_prefix + "test_custom_videos.json",
#         json_prefix + "test_2.json"
#         # "test_lsvq.json",
#         # json_prefix + "test_lsvq_1080p.json",
#         # json_prefix + "konvid.json",
#         # json_prefix + "maxwell_test.json",
#     ]

#     os.makedirs(f"results/{args.model_path}/", exist_ok=True)


#     conv_mode = "mplug_owl2"

#     inp = "How would you rate the quality of this video?"

#     conv = conv_templates[conv_mode].copy()
#     inp =  inp + "\n" + DEFAULT_IMAGE_TOKEN
#     conv.append_message(conv.roles[0], inp)
#     image = None

#     conv.append_message(conv.roles[1], None)
#     prompt = conv.get_prompt() + " The quality of the video is"

#     toks = ["good", "poor", "high", "fair", "low", "excellent", "bad", "fine", "moderate",  "decent", "average", "medium", "acceptable"]
#     print(toks)
#     ids_ = [id_[1] for id_ in tokenizer(toks)["input_ids"]]
#     print(ids_)

#     input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(args.device)

#     for image_path, json_ in zip(image_paths, jsons):
#         with open(json_) as f:
#             iqadata = json.load(f)
#             prs, gts = [], []
#             for i, llddata in enumerate(iqadata):
#                 try:
#                     try:
#                         filename = llddata["img_path"]
#                     except:
#                         filename = llddata["image"]
#                     llddata["logits"] = defaultdict(float)

#                     image = load_video(image_path + filename)
#                     def expand2square(pil_img, background_color):
#                             width, height = pil_img.size
#                             if width == height:
#                                 return pil_img
#                             elif width > height:
#                                 result = Image.new(pil_img.mode, (width, width), background_color)
#                                 result.paste(pil_img, (0, (width - height) // 2))
#                                 return result
#                             else:
#                                 result = Image.new(pil_img.mode, (height, height), background_color)
#                                 result.paste(pil_img, ((height - width) // 2, 0))
#                                 return result
#                     image = [expand2square(img, tuple(int(x*255) for x in image_processor.image_mean)) for img in image]
#                     image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().to(args.device)

#                     if True:
#                         with torch.inference_mode():
#                             output_logits = model(input_ids,
#                                 images=[image_tensor])["logits"][:,-1]
#                             for tok, id_ in zip(toks, ids_):
#                                 llddata["logits"][tok] += output_logits.mean(0)[id_].item()
#                             llddata["score"] = wa5(llddata["logits"])
#                             # print(llddata)
#                             prs.append(llddata["score"])

#                             gts.append(llddata["gt_score"])
#                             print("The imageName is" , llddata["img_path"], "and the score is ", llddata["score"] )

#                             json_ = json_.replace("combined/", "combined-")
#                             with open(f"results/{args.model_path}/{json_.split('/')[-1]}", "a") as wf:
#                                 json.dump(llddata, wf)

#                     if i > 0 and i % 200 == 0:
#                         print(spearmanr(prs,gts)[0], pearsonr(prs,gts)[0])
#                 except:
#                     continue
#             print("Spearmanr", spearmanr(prs,gts)[0], "Pearson", pearsonr(prs,gts)[0])


# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--model-path", type=str, default="q-future/one-align")
#     parser.add_argument("--model-base", type=str, default=None)
#     parser.add_argument("--device", type=str, default="cuda:0")
#     parser.add_argument("--conv-mode", type=str, default=None)
#     parser.add_argument("--temperature", type=float, default=0.2)
#     parser.add_argument("--max-new-tokens", type=int, default=512)
#     parser.add_argument("--load-8bit", action="store_true")
#     parser.add_argument("--load-4bit", action="store_true")
#     parser.add_argument("--debug", action="store_true")
#     parser.add_argument("--image-aspect-ratio", type=str, default='pad')
#     args = parser.parse_args()
#     main(args)


Overwriting /content/Q-Align/q_align/evaluate/vqa_eval.py


In [ ]:
import sys
import torch
sys.path.append('/content/Q-Align')

if 'q_align.evaluate.vqa_eval' in sys.modules:
    del sys.modules['q_align.evaluate.vqa_eval']

from q_align.evaluate.vqa_eval import main
from types import SimpleNamespace

args = SimpleNamespace(
    model_path= "/content/Q-Align/checkpoints",  # Use the path we set earlier
    model_base=None,  # Set this to None if loading a full model
    device="cuda" if torch.cuda.is_available() else "cpu",
    conv_mode=None,
    temperature=0.2,
    max_new_tokens=512,
    load_8bit=False,
    load_4bit=False,
    debug=False,
    image_aspect_ratio='pad'
)

main(args)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

['good', 'poor', 'high', 'fair', 'low', 'excellent', 'bad', 'fine', 'moderate', 'decent', 'average', 'medium', 'acceptable']
[1781, 6460, 1880, 6534, 4482, 15129, 4319, 2691, 17768, 27189, 6588, 18350, 22691]
The imageName is SpiderMan_Bad.mp4 and the score is  0.29016397734543
The imageName is Mask_good.mp4 and the score is  0.9134963911264358
The imageName is Extra_Hand_CHeck.mp4 and the score is  0.9158179767528758
The imageName is Decent_Girl_Face.mp4 and the score is  0.9474834830497143
The imageName is Good_Product.mp4 and the score is  0.9404160025476003
The imageName is Good_Gorl_Face.mp4 and the score is  0.8527966170758993
The imageName is Good_GirlWalking.mp4 and the score is  0.8743089204025365
The imageName is Not Good_Hand.mp4 and the score is  0.9548868702266509
The imageName is Good_FIngers.mp4 and the score is  0.9349286112255262
The imageName is Girl_Bad.mp4 and the score is  0.9317214695741906
The imageName is i2v.mp4 and the score is  0.9464109278223132
The imageNam